# Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json

from collections import defaultdict

# FIRST DATA FILE

### Read in data

In [ ]:
filename = "/Users/gandalf/Documents/data/data_users.csv"

user_df = pd.read_csv(filename, parse_dates=['created', 
                                        'updated', 
                                        'available', 
                                        'birthday', 
                                        'lastActive'])

user_df = user_df.fillna({'legacyId' : 'None',
                'about'    : '', 
                'birthday' : pd.to_datetime('1899-01-01'), 
                'latitude' : 0, 
                'longitude': 0})

col_to_drop = ['hashedPassword','firstName','lastName','username']
user_df = user_df.drop(col_to_drop, axis=1)

user_df.head(2)
               
# get just user email end

# SECOND DATA FILE

### Read in data

In [ ]:
filename = "/Users/gandalf/Documents/data/data_messages.json"
json_data=open(filename).read()
data = json.loads(json_data)

### Create first data frame (messages)

In [ ]:
column_names = ['message_id',       # Added, unique for each row
               'conversation_id',   # Added, identifies the conversation, not unique
               'uid',               # message sender
               'read',              # true/false value
               'readBy',            # message recipient(s)
               'number_of_people',  # Added, num of people in conversation
               'text_length',       # Added, length of text
               'timestamp',         # time
               'imageURL',          # image url
               'emailAttempted',    # ?
               'emailed']           # ?

mi,ci,ui,rd,rb,np,tl,ts,iu,ea,em = [],[],[],[],[],[],[],[],[],[],[]
  
for conversation_id, conversation_data in data["conversations"].items():
    for message_id, message_data in conversation_data.items():
        information = defaultdict(lambda: '', message_data)
        ci.append(conversation_id)
        mi.append(message_id)
        ui.append(information['uid'])
        rd.append(information['read'])
        rb.append(information['readBy'])
        np.append(1+len(information['readBy']))
        tl.append(len(information['text']))
        ts.append(information['timestamp'])
        iu.append(information['imageURL'])
        ea.append(information['emailAttempted'])
        em.append(information['emailed'])

message_df = pd.DataFrame([mi,ci,ui,rd,rb,np,tl,ts,iu,ea,em]).T
message_df.columns=column_names

message_df.head(2)

### Create second data frame

In [ ]:
column_names = ['user_id',       # user id
               'first_ten',      # first ten of the 20 digit key
               'last_ten',       # last ten of the 20 digit key (prob userid)
               'lastMessageId']  # message id

ui,ft,lt,lm = [],[],[],[]

for user_id, user_data in data['users'].items():
    for key, value in user_data['conversations'].items():
        ui.append(user_id)
        ft.append(key[:10])
        lt.append(key[10:])
        lm.append(value['lastMessageId'])

lastmessage_df = pd.DataFrame([ui,ft,lt,lm]).T
lastmessage_df.columns=column_names

lastmessage_df.head(2)

# SAVE AS PICKLE

In [ ]:
path_users = "/Users/gandalf/Documents/data/data_users.pkl"
path_message = "/Users/gandalf/Documents/data/data_message.pkl"
path_lastmessage = "/Users/gandalf/Documents/data/data_lastmessage.pkl"
user_df.to_pickle(path_users)
message_df.to_pickle(path_message)
lastmessage_df.to_pickle(path_lastmessage)